In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import normalize


from joblib import parallel_backend

import xgboost as xgb

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import math

import pickle

import itertools
from tqdm import tqdm
from scipy import sparse


In [2]:
outpatient = pd.read_pickle("../data/raw/outpatient.pkl")
# outpatient = ingest.get_cache_data("Outpatient", "Outpatient.pkl")

In [3]:
outpatient.dropna(subset=['CLM_PMT_AMT'], inplace=True)
outpatient = outpatient[['CLM_PMT_AMT']]

In [5]:
with open('../data/features/feature_vectors_dictionary-truncated.pkl', 'rb') as handle:
    feature_dictionary = pickle.load(handle)

for met_set in feature_dictionary.keys():
    print(met_set)
    feature_dictionary[met_set] = normalize(sparse.csr_matrix(feature_dictionary[met_set]))
    print(np.shape(feature_dictionary[met_set]))

BIRTH_DATE
(792562, 1)
SEX
(792562, 1)
State
(792562, 52)
County
(792562, 309)
clm_dates
(792562, 1)
provider
(792562, 293)
provider_count
(792562, 1)
DGNS_CD
(792562, 999)
DGNS_CD_count
(792562, 1)
PRDCR_CD
(792562, 185)
PRDCR_CD_count
(792562, 1)
HCPCS_CD
(792562, 725)
HCPCS_CD_count
(792562, 1)


In [6]:
def run_model(X, y, model, grid, feature_names):

       ### Runs the model and returns MSE and R2 values and corresponding data as a dataframe

       ### Takes in four arugments
       ### X is the features to that model will be trained on as a sparse matrix
       ### y is the scalar that is to be predicted as a list
       ### model is the regression model as a model object
       ### grid is a dictionary of model hyperparameters

       ### returns a dataframe of results from the grid search

       scoring = {'mean_squared_error': make_scorer(mean_squared_error), 'r2_score': make_scorer(r2_score)}
       with parallel_backend('threading', n_jobs=2):
              grid_model = GridSearchCV(model, grid, scoring=scoring, refit=False, return_train_score=True, error_score="raise").fit(X.todense(), y)
       
       print(grid_model.cv_results_)

       df=pd.DataFrame.from_dict(grid_model.cv_results_)
       
       return df

In [7]:
def iterate_feature_sets(model, feature_dict, grid, feature_sets):

    ### Creates and runs the specified model with each comboniation of features sets from 1 to k length features and stores the results of the models into a dictionary

    ### Takes in four arguments
    ### model is the regression model as a model object
    ### feature_dict is the dictionary of feature sets
    ### grid is a dictionary of model hyperparameters
    ### feature_sets dicates how what feature sets will be returned, a list of list of strings corresponding to the keys in the dictionary
    ### (continued) all would return all combinations for 1 through k, where  k is the number of keys in feature_dict

    ### returns a dictionary of results

    results = {}

    for feature_list in feature_sets:
        if feature_list == 'all':

            for n_met in tqdm(range(len(feature_dict))):
                for met_set in list(itertools.combinations(feature_dict.keys(), n_met+1)):
                    for n_met_set in range(len(met_set)):
                        if n_met_set == 0:
                            X = feature_dict[met_set[n_met_set]]
                        else:
                            X  = sparse.hstack((X,feature_dict[met_set[n_met_set]]), format='csr')
                    answer = run_model(X, outpatient['CLM_PMT_AMT'].values.tolist(), model, grid, met_set)
                    results[met_set] = answer

        else:
            X = None
            Name = ()
            for i in feature_list:
                if X == None:
                    X = feature_dict[i]
                    Name += (i,)
                else:
                    X = sparse.hstack((X, feature_dict[i]), format='csr')
                    Name += (i,)
            answer = run_model(X, outpatient['CLM_PMT_AMT'].values.tolist(), model, grid, feature_list)
            results[Name] = answer
        
    return results

In [8]:
def write_results(results_dict, filename):

    ### Takes in two arguements
    ### results_dict is the dictionary of feature sets
    ### file name is the wanted file name as a string

    ### Returns None, but writes a file to the current directory


    df = pd.DataFrame()

    for r_key in results_dict.keys():
        temp_df = results_dict[r_key]
        temp_df['features'] = str(r_key)
        df = pd.concat((df, temp_df), ignore_index=True)

    df.to_csv(filename)

    return None

# Initial Gradient Boosted Results
### Purpose is to find which features sets matter before hyperparameter tuning

In [9]:
GBRModel = HistGradientBoostingRegressor(random_state=42)
write_results(iterate_feature_sets(GBRModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'HCPCS_CD'],
['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'HCPCS_CD'],
['provider', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'HCPCS_CD'],
['HCPCS_CD'],
['PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD'],
['clm_dates', 'provider', 'DGNS_CD'],
['clm_dates', 'DGNS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD', 'PRDCR_CD'],
['provider', 'DGNS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD'],
['clm_dates'],
['clm_dates', 'PRDCR_CD'],
['clm_dates', 'provider'],
['clm_dates', 'provider', 'PRDCR_CD'],
['provider', 'PRDCR_CD'],
['PRDCR_CD'],
['provider']]), 'Initial_GBRModel.csv')

TypeError: np.matrix is not supported. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

# Initial Stochastic Gradient Descent Results
### Purpose is to find which features set's matter before hyperparameter tuning

In [9]:
"""
SGDModel = SGDRegressor(random_state=42)
write_results(iterate_feature_sets(SGDModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'HCPCS_CD'],
['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'HCPCS_CD'],
['provider', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'HCPCS_CD'],
['HCPCS_CD'],
['PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD'],
['clm_dates', 'provider', 'DGNS_CD'],
['clm_dates', 'DGNS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD', 'PRDCR_CD'],
['provider', 'DGNS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD'],
['clm_dates'],
['clm_dates', 'PRDCR_CD'],
['clm_dates', 'provider'],
['clm_dates', 'provider', 'PRDCR_CD'],
['provider', 'PRDCR_CD'],
['PRDCR_CD'],
['provider']]), 'Initial_SGDModel.csv')
"""

"\nSGDModel = SGDRegressor(random_state=42)\nwrite_results(iterate_feature_sets(SGDModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'HCPCS_CD'],\n['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],\n['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'DGNS_CD', 'HCPCS_CD'],\n['DGNS_CD', 'HCPCS_CD'],\n['clm_dates', 'HCPCS_CD'],\n['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'provider', 'HCPCS_CD'],\n['provider', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'HCPCS_CD'],\n['HCPCS_CD'],\n['PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'PRDCR_CD'],\n['clm_dates', 'provider', 'DGNS_CD'],\n['clm_dates', 'DGNS_CD'],\n['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],\n['DGNS_CD', 'PRDCR_CD'],\n['provider', 'DGNS_CD'],\n['provider', 'DGNS_CD', 'PRDCR_CD'],\n['DGNS_CD'],\n[

# Initial Elastic-Net Results
### Purpose is to find which features set's matter before hyperparameter tuning

In [10]:
"""
ENModel = ElasticNet(random_state=42)
write_results(iterate_feature_sets(ENModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'HCPCS_CD'],
['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'HCPCS_CD'],
['provider', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'HCPCS_CD'],
['HCPCS_CD'],
['PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD'],
['clm_dates', 'provider', 'DGNS_CD'],
['clm_dates', 'DGNS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD', 'PRDCR_CD'],
['provider', 'DGNS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD'],
['clm_dates'],
['clm_dates', 'PRDCR_CD'],
['clm_dates', 'provider'],
['clm_dates', 'provider', 'PRDCR_CD'],
['provider', 'PRDCR_CD'],
['PRDCR_CD'],
['provider']]), 'Initial_ENModel.csv')
"""

"\nENModel = ElasticNet(random_state=42)\nwrite_results(iterate_feature_sets(ENModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'HCPCS_CD'],\n['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],\n['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'DGNS_CD', 'HCPCS_CD'],\n['DGNS_CD', 'HCPCS_CD'],\n['clm_dates', 'HCPCS_CD'],\n['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'provider', 'HCPCS_CD'],\n['provider', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'HCPCS_CD'],\n['HCPCS_CD'],\n['PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'PRDCR_CD'],\n['clm_dates', 'provider', 'DGNS_CD'],\n['clm_dates', 'DGNS_CD'],\n['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],\n['DGNS_CD', 'PRDCR_CD'],\n['provider', 'DGNS_CD'],\n['provider', 'DGNS_CD', 'PRDCR_CD'],\n['DGNS_CD'],\n['clm

# Initial XGBoost Results
### Purpose is to find which features set's matter before hyperparameter tuning

In [11]:
"""
XGBModel = xgb.(random_state=42)
write_results(iterate_feature_sets(XGBModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'DGNS_CD', 'HCPCS_CD'],
['DGNS_CD', 'HCPCS_CD'],
['clm_dates', 'HCPCS_CD'],
['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'provider', 'HCPCS_CD'],
['provider', 'PRDCR_CD', 'HCPCS_CD'],
['provider', 'HCPCS_CD'],
['HCPCS_CD'],
['PRDCR_CD', 'HCPCS_CD'],
['clm_dates', 'DGNS_CD', 'PRDCR_CD'],
['clm_dates', 'provider', 'DGNS_CD'],
['clm_dates', 'DGNS_CD'],
['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD', 'PRDCR_CD'],
['provider', 'DGNS_CD'],
['provider', 'DGNS_CD', 'PRDCR_CD'],
['DGNS_CD'],
['clm_dates'],
['clm_dates', 'PRDCR_CD'],
['clm_dates', 'provider'],
['clm_dates', 'provider', 'PRDCR_CD'],
['provider', 'PRDCR_CD'],
['PRDCR_CD'],
['provider']]), 'InitiaXGBModel.csv')
"""

"\nXGBModel = xgb.(random_state=42)\nwrite_results(iterate_feature_sets(XGBModel, feature_dictionary, {}, [['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'HCPCS_CD'],\n['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD'],\n['DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'DGNS_CD', 'HCPCS_CD'],\n['DGNS_CD', 'HCPCS_CD'],\n['clm_dates', 'HCPCS_CD'],\n['clm_dates', 'provider', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'provider', 'HCPCS_CD'],\n['provider', 'PRDCR_CD', 'HCPCS_CD'],\n['provider', 'HCPCS_CD'],\n['HCPCS_CD'],\n['PRDCR_CD', 'HCPCS_CD'],\n['clm_dates', 'DGNS_CD', 'PRDCR_CD'],\n['clm_dates', 'provider', 'DGNS_CD'],\n['clm_dates', 'DGNS_CD'],\n['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD'],\n['DGNS_CD', 'PRDCR_CD'],\n['provider', 'DGNS_CD'],\n['provider', 'DGNS_CD', 'PRDCR_CD'],\n['DGNS_CD'],\n['clm_dat

# Grid Search Gradient Boosted Results
### Purpose is to find which hyperparameters perform the best on the top three feature selections + our demographic information

In [12]:
"""
grid_dict = {#'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
    #'max_iter': [1, 2, 4, 8, 16, 32, 64, 100, 200],
    'max_leaf_nodes': [2, 4, 6],
    #'max_depth': [np.linspace(1, 32, 32, endpoint=True)],
    #'min_samples_leaf': [np.linspace(0.1, 0.5, 5, endpoint=True)]
    }


GBRModel = HistGradientBoostingRegressor(random_state=42)
for search in grid_dict.keys():
    write_results(iterate_feature_sets(GBRModel, feature_dictionary, {search: grid_dict[search]}, [
        ['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
        ['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
        ['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_GBRModel2.csv')
"""

"\ngrid_dict = {#'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],\n    #'max_iter': [1, 2, 4, 8, 16, 32, 64, 100, 200],\n    'max_leaf_nodes': [2, 4, 6],\n    #'max_depth': [np.linspace(1, 32, 32, endpoint=True)],\n    #'min_samples_leaf': [np.linspace(0.1, 0.5, 5, endpoint=True)]\n    }\n\n\nGBRModel = HistGradientBoostingRegressor(random_state=42)\nfor search in grid_dict.keys():\n    write_results(iterate_feature_sets(GBRModel, feature_dictionary, {search: grid_dict[search]}, [\n        ['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n        ['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n        ['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_GBRModel2.csv')\n"

# Grid Search Stochastic Gradient Descent Results
### Purpose is to find which hyperparameters perform the best on the top three feature selections + our demographic information

In [13]:
"""
grid_dict = {'loss': ['squared_loss', 'squared_epsilon_insensitive'],
    'penalty': ['l1', 'l2'],
    'alpha': [.001, .0001, .00001],
    'max_iter': [1000, 5000, 10000]}

SGDModel = SGDRegressor(random_state=42)
for search in grid_dict.keys():
    write_results(iterate_feature_sets(SGDModel, feature_dictionary, {search: grid_dict[search]}, [
['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_SGDModel.csv')
"""

"\ngrid_dict = {'loss': ['squared_loss', 'squared_epsilon_insensitive'],\n    'penalty': ['l1', 'l2'],\n    'alpha': [.001, .0001, .00001],\n    'max_iter': [1000, 5000, 10000]}\n\nSGDModel = SGDRegressor(random_state=42)\nfor search in grid_dict.keys():\n    write_results(iterate_feature_sets(SGDModel, feature_dictionary, {search: grid_dict[search]}, [\n['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_SGDModel.csv')\n"

# Grid Search Elastic Net Results
### Purpose is to find which hyperparameters perform the best on the top three feature selections + our demographic information

In [14]:
"""
grid_dict = {'alpha': [.0001, .001, .01],
            'l1_ratio': [.1, .2, .3, .4, .5, .6, .7, .8, .9]}

ENModel = ElasticNet(random_state=42)
for search in grid_dict.keys():
    write_results(iterate_feature_sets(ENModel, feature_dictionary, {search: grid_dict[search]}, [
['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'PRDCR_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_ENModel.csv')
"""

"\ngrid_dict = {'alpha': [.0001, .001, .01],\n            'l1_ratio': [.1, .2, .3, .4, .5, .6, .7, .8, .9]}\n\nENModel = ElasticNet(random_state=42)\nfor search in grid_dict.keys():\n    write_results(iterate_feature_sets(ENModel, feature_dictionary, {search: grid_dict[search]}, [\n['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'PRDCR_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n['clm_dates', 'provider', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_ENModel.csv')\n"

# Grid Search XGBoost Results
### Purpose is to find which hyperparameters perform the best on the top three feature selections + our demographic information

In [15]:
"""
grid_dict = {'n_estimators': [3, 6, 9, 12, 15, 18],
    'tree_method': ['approx', 'hist'],
    'max_depth': [5,10,15,20,25,30,35]}


XGBModel = xgb.XGBRFRegressor(random_state=42)
for search in grid_dict.keys():
    write_results(iterate_feature_sets(XGBModel, feature_dictionary, {search: grid_dict[search]}, [
        ['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
        ['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],
        ['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_XGBModel.csv')
"""

"\ngrid_dict = {'n_estimators': [3, 6, 9, 12, 15, 18],\n    'tree_method': ['approx', 'hist'],\n    'max_depth': [5,10,15,20,25,30,35]}\n\n\nXGBModel = xgb.XGBRFRegressor(random_state=42)\nfor search in grid_dict.keys():\n    write_results(iterate_feature_sets(XGBModel, feature_dictionary, {search: grid_dict[search]}, [\n        ['clm_dates', 'provider', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n        ['clm_dates', 'DGNS_CD', 'PRDCR_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County'],\n        ['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), search+'_GridSearch_XGBModel.csv')\n"

# Final Histogram Gradient Boosted NonPCA Supervised Run
### See how the model performs on the dataset without PCA

In [16]:

grid_dict = {'learning_rate': [0.25],
    'max_iter': [200],
    #wanted to go a little higher on leaf nodes since there seemed to be more to gain
    'max_leaf_nodes': [10],
    'max_depth': [18]
    #results were inconclusive for minimum samples, going to leave as default
    }


GBRModel = HistGradientBoostingRegressor(random_state=42)
write_results(iterate_feature_sets(GBRModel, feature_dictionary, grid_dict, [
    # picked because of speed and results
    ['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), 'NPCA_Final_GBRModel.csv')


# Final Stochastic Gradient Descent NonPCA Supervised Run
### See how the model performs on the dataset without PCA

In [ ]:
grid_dict = {'loss': ['squared_epsilon_insensitive'],
    'penalty': ['l1'],
    'alpha': [.0001],
    'max_iter': [1000]}

SGDModel = SGDRegressor(random_state=42)
write_results(iterate_feature_sets(SGDModel, feature_dictionary, grid_dict, [
['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), 'NPCA_Final_SGDModel.csv')

NameError: name 'SGDRegressor' is not defined

# Final Elastic Net NonPCA Supervised Run
### See how the model performs on the dataset without PCA

In [ ]:
grid_dict = {'alpha': [.0001],
    'l1_ratio': [.9]}

ENModel = ElasticNet(random_state=42)
write_results(iterate_feature_sets(ENModel, feature_dictionary, grid_dict, [
['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), 'NPCA_Final_ENModel.csv')

{'mean_fit_time': array([1313.70082536]), 'std_fit_time': array([173.4870673]), 'mean_score_time': array([3.42845478]), 'std_score_time': array([0.60479199]), 'param_alpha': masked_array(data=[0.0001],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_l1_ratio': masked_array(data=[0.9],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.0001, 'l1_ratio': 0.9}], 'split0_test_mean_squared_error': array([234575.36121982]), 'split1_test_mean_squared_error': array([236388.11086495]), 'split2_test_mean_squared_error': array([237540.75215016]), 'split3_test_mean_squared_error': array([243170.68852935]), 'split4_test_mean_squared_error': array([242236.20190961]), 'mean_test_mean_squared_error': array([238782.22293478]), 'std_test_mean_squared_error': array([3351.39698735]), 'rank_test_mean_squared_error': array([1]), 'split0_train_mean_squared_error': array([238319.35366125]), 'split1_train_mean_squared_error': 

# Final XGBoost NonPCA Supervised Run
### See how the model performs on the dataset without PCA

In [ ]:
grid_dict = {'max_depth': [10],
    'number_of_estimators': [9],
    'tree_method': ['hist']}

XGBModel = xgb.XGBRFRegressor(random_state=42)
write_results(iterate_feature_sets(XGBModel, feature_dictionary, grid_dict, [
['clm_dates', 'DGNS_CD', 'HCPCS_CD', 'BIRTH_DATE', 'SEX', 'State', 'County']]), 'NPCA_Final_XGBModel.csv')

{'mean_fit_time': array([255.17862582]), 'std_fit_time': array([41.15142138]), 'mean_score_time': array([2.32038045]), 'std_score_time': array([0.69958266]), 'param_max_depth': masked_array(data=[10],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_number_of_estimators': masked_array(data=[9],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_tree_method': masked_array(data=['hist'],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 10, 'number_of_estimators': 9, 'tree_method': 'hist'}], 'split0_test_mean_squared_error': array([229596.14047328]), 'split1_test_mean_squared_error': array([229405.51322255]), 'split2_test_mean_squared_error': array([231170.87972658]), 'split3_test_mean_squared_error': array([235786.80856708]), 'split4_test_mean_squared_error': array([234357.414672]), 'mean_test_mean_squared_error': array([232063.3513323]), 'std_test_mean_squared_error':